# Calculating change over time


## imports


In [ ]:
import leafmap
import requests
import rasterio as rio
import glob

## searching for data


In [ ]:
url = "https://earth-search.aws.element84.com/v1/"
collection = "sentinel-2-l2a"
time_range = "2023-08-01/2023-08-31"

In [ ]:
# bbox for dallas metro
bbox = [
    -97.06213756027009,
    32.97324551867027,
    -96.46807822577594,
    33.3578329610085,
]

In [ ]:
search = leafmap.stac_search(
    url=url,
    max_items=10,
    collections=[collection],
    bbox=bbox,
    datetime=time_range,
    query={"eo:cloud_cover": {"lt": 20}},
    sortby=[{"field": "properties.eo:cloud_cover", "direction": "asc"}],
    get_links=True,
)
search

In [ ]:
search_gdf = leafmap.stac_search(
    url=url,
    max_items=10,
    collections=[collection],
    bbox=bbox,
    datetime=time_range,
    query={"eo:cloud_cover": {"lt": 20}},
    sortby=[{"field": "properties.eo:cloud_cover", "direction": "asc"}],
    get_gdf=True,
)

In [ ]:
search_gdf.columns

## exploring and filtering search results


In [ ]:
search_gdf.plot("mgrs:grid_square", alpha=0.25)

In [ ]:
search_gdf["s2:granule_id"].value_counts()

In [ ]:
search_gdf["area"] = search_gdf.geometry.area

In [ ]:
search_gdf.sort_values("area", ascending=False, inplace=True)

In [ ]:
search_gdf

In [ ]:
search_gdf.drop_duplicates("mgrs:grid_square", inplace=True)

In [ ]:
search_gdf.iloc[0:4].plot("s2:granule_id", alpha=0.5)

In [ ]:
# # create a query searching for unique datastrip ids based on the above
q = {"s2:granule_id": {"in": search_gdf["s2:granule_id"].iloc[0:4].unique().tolist()}}

search_gdf2 = leafmap.stac_search(
    url=url,
    max_items=4,
    collections=[collection],
    bbox=bbox,
    datetime=time_range,
    query=q,
    get_gdf=True,
)

In [ ]:
search_gdf2

In [ ]:
search_gdf2.plot("mgrs:grid_square", alpha=0.5)

In [ ]:
search

In [ ]:
search_gdf2 = leafmap.stac_search(
    url=url,
    max_items=4,
    collections=[collection],
    bbox=bbox,
    datetime=time_range,
    query=q,
    get_links=True,
)

In [ ]:
search_gdf2

### plot to accentuate paved and non-paved divide


In [ ]:
m = leafmap.Map()

for layer in search_gdf2:
    m.add_stac_layer(layer, bands=["nir", "red", "green"], name=layer.split("/")[-1])
m

### download files for analysis


In [ ]:
def get_raster_band_urls(item: str, bands: list | None = None):
    available_bands = leafmap.stac_bands(item)
    stac = requests.get(item).json()
    band_urls = {x: stac["assets"][x]["href"] for x in available_bands}

    # if bands, only return bands in list
    if bands:
        band_urls = {x: band_urls[x] for x in bands if x in band_urls}

    return band_urls


def download_stac_layers(layers, out_dir, bands=None):
    for layer in layers:
        band_urls = get_raster_band_urls(layer, bands)
        for band, url in band_urls.items():
            out_file = f"{out_dir}/{layer.split('/')[-1]}_{band}.tif"
            leafmap.download_file(url, out_file, overwrite=True)


def get_stac_crs(item):
    stac = requests.get(item).json()
    return stac["properties"]["proj:epsg"]

In [ ]:
stac_crs = get_stac_crs(search_gdf2[0])

In [ ]:
stac_crs

In [ ]:
download_stac_layers(
    search_gdf2, "../Data/stac/dallas", bands=["nir", "red", "green", "blue"]
)

In [ ]:
# mosaic files based on band name


def mosaic_by_band(dir, bands, crs):
    mosaics = {}
    for band in bands:
        files = glob.glob(f"{dir}/*{band}.tif")

        out_file = f"{dir}/mosaic_{band}.tif"
        raster_data = [rio.open(f) for f in files]

        mosaic, out_trans = rio.merge.merge(raster_data)
        out_meta = raster_data[0].meta.copy()
        out_meta.update(
            {
                "driver": "GTiff",
                "height": mosaic.shape[1],
                "width": mosaic.shape[2],
                "transform": out_trans,
                "crs": f"epsg:{crs}",
            }
        )

        with rio.open(out_file, "w", **out_meta) as dest:
            dest.write(mosaic)

        mosaics[band] = out_file

    return mosaics

In [ ]:
mosaic_bands = mosaic_by_band(
    "../Data/stac/dallas", ["nir", "red", "green", "blue"], stac_crs
)

In [ ]:
mosaic_bands

In [ ]:
# load, calculate ndvi, visualize ndvi
# segment based on visual spectrum, classify
# repeat whole process for past image (in a more programmatic way)
# diff the two to see what change has occurred

In [ ]:
nir = rio.open(mosaic_bands["nir"]).read(1).astype("float32")
red = rio.open(mosaic_bands["red"]).read(1).astype("float32")

### calculate NDVI


In [ ]:
ndvi = (nir - red) / (nir + red)

In [ ]:
import numpy as np

In [ ]:
# fill nan with -2, outside of the range but not so far outside
ndvi[np.isnan(ndvi)] = -2

In [ ]:
ndvi

In [ ]:
ndvi_image = leafmap.array_to_image(ndvi, source=mosaic_bands["nir"])

In [ ]:
m = leafmap.Map()
m.add_raster(ndvi_image, colormap="Greens", layer_name="NDVI")
m

## landsat


In [ ]:
url = "https://earth-search.aws.element84.com/v1/"
collection = "landsat-c2-l2"
time_range = "2023-08-01/2023-08-31"

In [ ]:
# bbox for dallas metro
bbox = [
    -97.06213756027009,
    32.97324551867027,
    -96.46807822577594,
    33.3578329610085,
]

In [ ]:
search = leafmap.stac_search(
    url=url,
    max_items=10,
    collections=[collection],
    bbox=bbox,
    datetime=time_range,
    query={"eo:cloud_cover": {"lt": 20}},
    sortby=[{"field": "properties.eo:cloud_cover", "direction": "asc"}],
    get_links=True,
)
search

In [ ]:
m = leafmap.Map()

# for layer in search[0]:
m.add_stac_layer(search[0], bands=["nir08", "red", "green"], name=layer.split("/")[-1])
m